In [1]:
import torch
import matplotlib.pyplot as plt
import torch_dct as dct
from lib.tools import *
from lib.decomposition import *
from lib.jpeg import *
from lib.sketchml import *
from lib.regression import *
import os
import pandas as pd

In [2]:
# tensor_dir = "../experiments_football/tensors/"
layer =60
tensor_dir = "./test_tensors/coco_tensor_yolov3/layer_"+str(layer)+"/"
output_dir = "./measurements/coco_test_diff/"

test_tensors = os.listdir(tensor_dir)

In [3]:
# def sort_fun(tensor_name):
#     return int(tensor_name.split("_")[2].split(".")[0])

# test_tensors.sort(key=sort_fun)

In [4]:
## Measurements
split_layer = []

tensor_rank=[]
tensor_sparsity=[]
tensor_regularity=[]
tensor_pictoriality=[]
tensor_decomposability=[]

jpeg_ratio=[]
jpeg_snr=[]
jpeg_mse=[]
decomposition_ratio=[]
decomposition_snr=[]
decomposition_mse=[]
regression_ratio=[]
regression_snr=[]
regression_mse=[]
sketchml_ratio=[]
sketchml_snr=[]
sketchml_mse=[]

In [ ]:
for i in range(19):
    print("************* Testing tensor: "+str(i)+"***************")
    split_layer.append(layer)
    tensor1 = torch.load(tensor_dir+test_tensors[i])
    tensor2 = torch.load(tensor_dir+test_tensors[i+1])
    tensor = tensor2 - tensor1
    tensor = tensor * (abs(tensor)>0.7)
    tensor = abs(tensor[0])


     # tensor_rank.append(calculate_cp_rank(tensor.numpy(),max_rank=10,tol=1e-4))
    tensor_rank.append(calculate_slice_avg_rank(tensor))
    tensor_sparsity.append(calculate_sparsity(tensor.numpy()))
    tensor_pictoriality.append(get_tensor_pictoriality(tensor))
    tensor_regularity.append(get_tensor_regularity(tensor))
    tensor_decomposability.append(get_tensor_decomposability(tensor))

    print("Rank: %f, Sparsity: %f, Picoriality: %f, Regularity: %f, Decomposability: %f" % (tensor_rank[-1],tensor_sparsity[-1],tensor_pictoriality[-1],tensor_regularity[-1],tensor_decomposability[-1]))

    ## Jpeg compression
    try:
        print("Doing JPEG compression")
        normalize_base, scale,zero_point, encoded_data, compressed_size_jpeg = compressor_jpeg_gray(tensor, 100)
        reconstructed_tensor_jpeg = decompressor_jpeg_gray(tensor.shape,normalize_base, scale,zero_point, encoded_data )
        ratio_jpeg = tensor.numel()*4/compressed_size_jpeg
        snr_jpeg = calculate_snr(tensor.size(),tensor.numpy(), reconstructed_tensor_jpeg.numpy())
        mse_jpeg = calculate_mse(tensor.numpy(), reconstructed_tensor_jpeg.numpy())
        jpeg_ratio.append(ratio_jpeg)
        jpeg_snr.append(snr_jpeg)
        jpeg_mse.append(mse_jpeg)
        print("Ratio: %f, SNR: %f, MSE: %f" % (ratio_jpeg,snr_jpeg,mse_jpeg))
    except:
        jpeg_ratio.append(-1)
        jpeg_snr.append(-1)
        jpeg_mse.append(-1)

    try:
        print("Doing Decomposition compression")
        factors, compressed_size_decomposition = compressor_decomposition_slice(tensor)
        reconstructed_tensor_decomposition = decompressor_decomposition_slice(tensor.shape,factors)
        snr_decomposition = calculate_snr(tensor.size(),tensor.numpy(), reconstructed_tensor_decomposition.numpy())
        mse_decomposition = calculate_mse(tensor.numpy(), reconstructed_tensor_decomposition.numpy())
        ratio_decomposition= tensor.numel()*4/compressed_size_decomposition
        decomposition_ratio.append(ratio_decomposition)
        decomposition_snr.append(snr_decomposition)
        decomposition_mse.append(mse_decomposition)
        print("Ratio: %f, SNR: %f, MSE: %f" % (ratio_decomposition,snr_decomposition,mse_decomposition))
    except:
        decomposition_ratio.append(-1)
        decomposition_snr.append(-1)
        decomposition_mse.append(-1)

    try:
        print("Doing Regression compression")
        factors, x_pos, x_neg, compressed_size_regression = compressor_regression(tensor, 3)
        reconstructed_tensor_regression = decompressor_regression(tensor.shape, factors, x_pos,x_neg )
        snr_regression = calculate_snr(tensor.size(),tensor.numpy(), reconstructed_tensor_regression.numpy())
        mse_regression = calculate_mse(tensor.numpy(), reconstructed_tensor_regression.numpy())
        ratio_regression = tensor.numel()*4 / compressed_size_regression
        regression_ratio.append(ratio_regression)
        regression_snr.append(snr_regression)
        regression_mse.append(mse_regression)
        print("Ratio: %f, SNR: %f, MSE: %f" % (ratio_regression,snr_regression,mse_regression))
    except:
        regression_ratio.append(-1)
        regression_snr.append(-1)
        regression_mse.append(-1)

    try:
        print("Doing SketchML compression")
        minmaxsketch,bucket_means, keys, compressed_size_sketchml = compressor_sketchml(tensor,256,0.3,3)
        reconstructed_tensor_sketchml = decompressor_sketchml(tensor.shape, minmaxsketch,bucket_means, keys)
        snr_sketchml = calculate_snr(tensor.size(),tensor.numpy(), reconstructed_tensor_sketchml.numpy())
        mse_sketchml = calculate_mse(tensor.numpy(), reconstructed_tensor_sketchml.numpy())
        ratio_sketchml = tensor.numel() *4 / compressed_size_sketchml
        sketchml_ratio.append(ratio_sketchml)
        sketchml_snr.append(snr_sketchml)
        sketchml_mse.append(mse_sketchml)
        print("Ratio: %f, SNR: %f, MSE: %f" % (ratio_sketchml,snr_sketchml,mse_sketchml))
    except:
        sketchml_ratio.append(-1)
        sketchml_snr.append(-1)
        sketchml_mse.append(-1)


************* Testing tensor: 0***************
Rank: 4.734375, Sparsity: 0.986172, Picoriality: 0.504511, Regularity: 0.665899, Decomposability: 0.635817
Doing JPEG compression
Ratio: 14.930526, SNR: 20.751123, MSE: 0.000183
Doing Decomposition compression
Ratio: 10.983498, SNR: 142.327213, MSE: 0.000000
Doing Regression compression


/home/rex/gitRepo/split_DNN_framework/characteristics/lib/regression.py:20: RankWarning: Polyfit may be poorly conditioned
  m = np.polyfit(x,y,polynominal)
/home/rex/gitRepo/split_DNN_framework/characteristics/lib/regression.py:20: RankWarning: Polyfit may be poorly conditioned
  m = np.polyfit(x,y,polynominal)
/home/rex/gitRepo/split_DNN_framework/characteristics/lib/regression.py:20: RankWarning: Polyfit may be poorly conditioned
  m = np.polyfit(x,y,polynominal)
/home/rex/gitRepo/split_DNN_framework/characteristics/lib/regression.py:20: RankWarning: Polyfit may be poorly conditioned
  m = np.polyfit(x,y,polynominal)
/home/rex/gitRepo/split_DNN_framework/characteristics/lib/regression.py:20: RankWarning: Polyfit may be poorly conditioned
  m = np.polyfit(x,y,polynominal)
/home/rex/gitRepo/split_DNN_framework/characteristics/lib/regression.py:20: RankWarning: Polyfit may be poorly conditioned
  m = np.polyfit(x,y,polynominal)
/home/rex/gitRepo/split_DNN_framework/characteristics/lib/

Ratio: 15.176357, SNR: 9.912780, MSE: 0.002218
Doing SketchML compression
Ratio: 139.505038, SNR: 6.498275, MSE: 0.004869
************* Testing tensor: 1***************
Rank: 7.859375, Sparsity: 0.973974, Picoriality: 0.445279, Regularity: 0.560537, Decomposability: 0.395433
Doing JPEG compression
Ratio: 7.453447, SNR: 19.275943, MSE: 0.000466
Doing Decomposition compression
Ratio: 6.616302, SNR: 140.581646, MSE: 0.000000
Doing Regression compression


/home/rex/gitRepo/split_DNN_framework/characteristics/lib/regression.py:20: RankWarning: Polyfit may be poorly conditioned
  m = np.polyfit(x,y,polynominal)
/home/rex/gitRepo/split_DNN_framework/characteristics/lib/regression.py:20: RankWarning: Polyfit may be poorly conditioned
  m = np.polyfit(x,y,polynominal)
/home/rex/gitRepo/split_DNN_framework/characteristics/lib/regression.py:20: RankWarning: Polyfit may be poorly conditioned
  m = np.polyfit(x,y,polynominal)
/home/rex/gitRepo/split_DNN_framework/characteristics/lib/regression.py:20: RankWarning: Polyfit may be poorly conditioned
  m = np.polyfit(x,y,polynominal)
/home/rex/gitRepo/split_DNN_framework/characteristics/lib/regression.py:20: RankWarning: Polyfit may be poorly conditioned
  m = np.polyfit(x,y,polynominal)
/home/rex/gitRepo/split_DNN_framework/characteristics/lib/regression.py:20: RankWarning: Polyfit may be poorly conditioned
  m = np.polyfit(x,y,polynominal)
/home/rex/gitRepo/split_DNN_framework/characteristics/lib/

Ratio: 14.939227, SNR: 9.851944, MSE: 0.004079
Doing SketchML compression
Ratio: 104.407843, SNR: 7.428048, MSE: 0.007128
************* Testing tensor: 2***************
Rank: 10.000000, Sparsity: 0.965791, Picoriality: 0.451142, Regularity: 0.528572, Decomposability: 0.230769
Doing JPEG compression
Ratio: 6.566406, SNR: 20.472531, MSE: 0.000432
Doing Decomposition compression
Ratio: 5.200000, SNR: 139.529886, MSE: 0.000000
Doing Regression compression


...

Ratio: 14.939227, SNR: 9.156714, MSE: 0.009506
Doing SketchML compression
Ratio: 68.085374, SNR: 7.479944, MSE: 0.013985
************* Testing tensor: 14***************
Rank: 13.269531, Sparsity: 0.947029, Picoriality: 0.403600, Regularity: 0.465878, Decomposability: -0.020733
Doing JPEG compression
Ratio: 6.032506, SNR: 22.021775, MSE: 0.000540
Doing Decomposition compression
Ratio: 3.918752, SNR: 137.915316, MSE: 0.000000
Doing Regression compression
Ratio: 14.939227, SNR: 9.024045, MSE: 0.010767
Doing SketchML compression
Ratio: 67.232323, SNR: 7.287288, MSE: 0.016060
************* Testing tensor: 15***************
Rank: 11.128906, Sparsity: 0.958222, Picoriality: 0.455113, Regularity: 0.500383, Decomposability: 0.143930
Doing JPEG compression
Ratio: 6.130813, SNR: 20.576019, MSE: 0.000583
Doing Decomposition compression
Ratio: 4.672517, SNR: 138.823519, MSE: 0.000000
Doing Regression compression


/home/rex/gitRepo/split_DNN_framework/characteristics/lib/regression.py:20: RankWarning: Polyfit may be poorly conditioned
  m = np.polyfit(x,y,polynominal)
/home/rex/gitRepo/split_DNN_framework/characteristics/lib/regression.py:20: RankWarning: Polyfit may be poorly conditioned
  m = np.polyfit(x,y,polynominal)
/home/rex/gitRepo/split_DNN_framework/characteristics/lib/regression.py:20: RankWarning: Polyfit may be poorly conditioned
  m = np.polyfit(x,y,polynominal)


Ratio: 14.939227, SNR: 9.419036, MSE: 0.007610
Doing SketchML compression
Ratio: 78.832024, SNR: 7.425306, MSE: 0.012044
************* Testing tensor: 16***************
Rank: 9.875000, Sparsity: 0.967415, Picoriality: 0.464641, Regularity: 0.535407, Decomposability: 0.240385
Doing JPEG compression
Ratio: 6.801313, SNR: 22.811754, MSE: 0.000260
Doing Decomposition compression
Ratio: 5.265823, SNR: 139.518719, MSE: 0.000000
Doing Regression compression


/home/rex/gitRepo/split_DNN_framework/characteristics/lib/regression.py:20: RankWarning: Polyfit may be poorly conditioned
  m = np.polyfit(x,y,polynominal)
/home/rex/gitRepo/split_DNN_framework/characteristics/lib/regression.py:20: RankWarning: Polyfit may be poorly conditioned
  m = np.polyfit(x,y,polynominal)
/home/rex/gitRepo/split_DNN_framework/characteristics/lib/regression.py:20: RankWarning: Polyfit may be poorly conditioned
  m = np.polyfit(x,y,polynominal)


Ratio: 14.939227, SNR: 9.363108, MSE: 0.005761
Doing SketchML compression
Ratio: 92.075552, SNR: 7.116182, MSE: 0.009665
************* Testing tensor: 17***************
Rank: 10.500000, Sparsity: 0.961111, Picoriality: 0.469926, Regularity: 0.527410, Decomposability: 0.192308
Doing JPEG compression
Ratio: 6.398048, SNR: 23.343434, MSE: 0.000265
Doing Decomposition compression


In [6]:
dict = {'split_layer':split_layer,
        'rank': tensor_rank, 
        'sparsity': tensor_sparsity, 
        'regularity': tensor_regularity,
        'pictoriality': tensor_pictoriality,
        'jpeg_ratio':jpeg_ratio,
        'jpeg_snr':jpeg_snr,
        'jpeg_mse':jpeg_mse,
        'decomposition_ratio':decomposition_ratio,
        'decomposition_snr':decomposition_snr,
        'decomposition_mse':decomposition_mse,
        'regression_ratio':regression_ratio,
        'regression_snr':regression_snr,
        'regression_mse':regression_mse,
        'sketchml_ratio':sketchml_ratio,
        'sketchml_snr':sketchml_snr,
        'sketchml_mse':sketchml_mse
        } 
   
df = pd.DataFrame(dict)

In [7]:
df.to_csv(output_dir+"layer"+str(layer)+"_diff.csv",index=False, header=True)